# Maintaining State Across Agent Runs

Adding memory to an agent

In [ ]:
!pip install llama-index -q -q

In [ ]:
!pip install tavily-python -q -q

In [ ]:
from tavily import AsyncTavilyClient
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.llms.openai import OpenAI
import os
from openai import OpenAI as OpenAIClient

raw_client = OpenAIClient()

API_KEY   = raw_client.api_key
API_BASE  = raw_client.base_url

tavily_api_key = os.environ["TAVILY_API_KEY"]

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=tavily_api_key)
    return str(await client.search(query))

llm = OpenAI(model="gpt-4o-mini", api_key=API_KEY, api_base=API_BASE)

workflow = AgentWorkflow.from_tools_or_functions(
    [search_web],
    llm=llm,
    system_prompt="You are a helpful assistant that answers questions. If you don't know the answer, you can search the web for information.",
)

In [ ]:
from llama_index.core.workflow import Context

# Configure a context to work with our workflow
ctx = Context(workflow)

response = await workflow.run(
    user_msg="My name is Rujul, nice to meet you!", ctx=ctx
)
print(str(response))

Nice to meet you, Rujul! How can I assist you today?


Now we can pass the same context to a second run, and it will remember what happened before:

In [ ]:
# Run the workflow again with the same context
response = await workflow.run(user_msg="What is my name?", ctx=ctx)
print(str(response))

Your name is Rujul.


In [ ]:
from llama_index.core.workflow import Context

# We've set the first parameter to be of type Context
async def set_name(ctx: Context, name: str) -> str:
    state = await ctx.get("state")
    state["name"] = name
    await ctx.set("state", state)
    return f"Name set to {name}"


stateful_workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

stateful_workflow_context = Context(stateful_workflow)

# This workflow will save data to the `state` variable in the context
response = await stateful_workflow.run(user_msg="My name is Rujul", ctx=stateful_workflow_context)
print(str(response))

# We can retrieve the value of `state` from the context directly
state = await stateful_workflow_context.get("state")
print("Name as stored in state: ",state["name"])

Your name has been set to Rujul.
Name as stored in state:  Rujul
